In [ ]:
from pydantic import BaseModel
from typing import Optional, Dict, Any
from enum import Enum

class Rez(BaseModel):
    result:Any = None
    flds:dict = None
    rez:Optional[BaseModel] = None # actually a Rez -- cannot define recursive classes
    extra: Optional[dict] = None
    info: Optional[dict] = None

class ArgMode(str, Enum):
    FIELD = "field"
    PRIOR = "prior"

def add_dicts(a:dict, b:dict):
    return {**a, **b}

class Foo(BaseModel):
    a:int=None
    b:str=None

    def fields(self):
        """
        Return only non-meta fields.
        """
        return set(name for name in self.__fields__ if self.__fields__[name].default != name)
    def field_values(self):
        return {f:self.__dict__[f] for f in self.fields() if self.__dict__[f] is not None}
    def compute_flds(self, flds:dict):
        selected_flds = {key:flds[key] for key in self.fields() if key not in self.field_values()}
        return add_dicts(self.field_values(), selected_flds)

    def execute(self, tag:str=None, rez:Rez=None):
        flds = compute_flds(rez.flds)
        return Rez(result=rez.result+1, args=rez.flds)



In [ ]:
f = Foo(a=3, b="sksks")
f.compute_flds({"b":"flingo"})

In [ ]:
from whendo.core.actions.file_action import FileAppendD
fa = FileAppendD(file="adf")
set(name for name in fa.__fields__ if fa.__fields__[name].default != name)
fa.__dict__

In [ ]:
a = {"a":"b"}
c = {"c":"d"}
{**a, **c}

In [ ]:
{"a":"b"}.update( {"c":"d"})

In [ ]:
from pydantic import BaseModel
from typing import Dict, Set

class Server(BaseModel):
    host:str
    port:int
    tags:Dict[str, Set[str]]

    """
    A server has a dictionary mapping keys to sets of tags. Keys
    serve as a way to qualify tags. It's up to the user to
    use this key:tag-set structure to some benefit.
    """
    def add_key_tag(self, key:str, tag:str):
        if key not in self.tags:
            self.tags[key] = set()
        self.tags[key].add(tag)
    
    def has_key_tag(self, key:str, tag:str):
        return key in self.tags and tag in self.tags[key]
    
    def delete_key_tag(self, key:str, tag:str):
        """
        Remove the tag from the key's set, removing
        an empty key.
        """
        tags = self.tags.get(key, None)
        if tags:
            tags.remove(tag)
            if len(tags) == 0:
                self.tags.pop(key)
    
    
    def delete_key(self, key:str):
        """
        Remove the dang key.
        """
        self.tags.pop(key)

    def delete_tag(self, tag:str):
        """
        Delete tag from all tag sets and remove
        all consequently empty tag sets.
        """
        to_remove = set()
        tags_copy = self.tags.copy()
        for key in tags_copy:
            if tag in self.tags_copy[key]
                self.tags[key].pop(tag)
                if len(self.tags[key]) == 0:
                    to_remove.append(key)
        for key in to_remove:
            self.tags.pop(key)
    
    def get_tags(self):
        return self.tags
    
    def get_tags_by_key(self, key:str):
        """
        Return a key's tags.

        for server in self.servers:
            for tag in server.get_tags_by_key("program_name"):
                Client(server=server).schedule_program(program_name=tag)
        """
        return self.tags.get(key, None)
    
    def get_keys(self, tags:Set[str], key_tag_mode:KeyTagMode=KeyTagMode.ANY):
        if key_tag_mode = KeyTagMode.ANY:
            """
            Return the keys having tag sets that intersect the supplied tags.
            """
            return set(key for key in self.tags if len(intersection(tags, self.tags[key])) > 0)
        if key_tag_mode = KeyTagMode.ALL:
            """
            Return the keys where the supplied tag set covers the key's tags.
            """
            return set(key for key in self.tags if len(difference(tags, self.tags[key])) == 0)
    
class KeyTagMode(str, Enum):
    ALL = "all"
    ANY = "any"

class Dispatcher(BaseModel):
    servers:Dict[str,Server]={}
    def add_server(self, server_name:str, server:Server):
        self.check_server_name(server_name, invert=True)
        self.servers[server_name] = server
    def set_server(self, server_name:str, server:Server):
        self.check_server_name(server_name)
        self.servers[server_name] = server
    def delete_server(self, server_name:str):
        self.check_server_name(server_name)
        self.servers.pop(server_name)
    def get_server(self, server_name:str):
        self.check_server_name(server_name)
        return self.servers[server_name]
    def get_servers(self):
        return self.servers
    def apply_to_server(self, action:Action, server_key_tags:Dict[str,Set[str]], key_tag_mode:KeyTagMode=KeyTagMode.ANY):
        


Dispatcher.apply_to_server(dispatcher_action=some_action, server_key_tags={"server_type":{"pivot"}})




In [ ]:
from pydantic import BaseModel
from typing import Optional, Dict, Any, List
from datetime import datetime


class ActionSchedule(BaseModel):
    name: str  # 'dispatcher' | program name
    scheduled: Dict[str, List[str]] = {}
    deferred: Dict[str, Dict[str, List[str]]] = {}
    expiring: Dict[str, Dict[str, List[str]]] = {}


class ActionSchedules(BaseModel):
    schedules: Dict[str, ActionSchedule] = {}


class DeferredProgram(BaseModel):
    program_name: str
    start: datetime
    stop: datetime


class DeferredPrograms(BaseModel):
    programs: Dict[str, DeferredProgram]

In [ ]:
class AddAction(DispatcherAction):
    action_name: str
    action: Action

    pass


add_pinstate = AddAction(action_name="foo", action=PinState(pin=25))
dispatcher.add_action("add_pinstate", add_pinstate)
ExecuteAction(action_name="add_pinstate").execute()
add_add_pinstate = AddAction(action_name="bar", action=add_pinstate)


class AddScheduler(DispatcherAction):
    scheduler_name: str
    scheduler: scheduler


add_immediately = AddScheduler(scheduler_name="immediately", scheduler=Immediately())

dispatcher.add_action("add_immediately", add_immediately)
dispatcher.execute_action("add_immediately")


class ExecuteAction(DispatcherAction):
    action_name: str


exec_action = ExecuteAction(action_name="")

In [ ]:
from pydantic import BaseModel, PrivateAttr
from threading import RLock
from typing import Callable


class SharedLock:
    lock = RLock()


class Shared(BaseModel):
    """
    This class provides in-memory shared data so that actions can communicate with each other during
    an api server's lifetime. Not meant to be persisted. Persistent shared data is a different feature.

    Usage:
        sh = Shareds.get('foo')
        def update(d:dict):
            d['a'] = 1
            return 1
        result = sh.apply(update)
        print(result, sh.copy_data())
    """

    # not treated as a model attr
    _data: dict = PrivateAttr(default_factory=dict)

    def copy_data(self):
        return self._data.copy()

    def apply(self, callable):
        """
        Transactionally applies callable to a copy of the data. A failure within
        the callable won't corrupt the dictionary.
        """
        with SharedLock.lock:
            copy = self._data.copy()
            result = callable(copy)
            self._data = copy
            return result


class Shareds:
    """
    A dictionary of instances of Shared.
    """

    singletons = {}

    @classmethod
    def get(cls, label: str) -> Shared:
        if label not in cls.singletons:
            cls.singletons[label] = Shared()
        return cls.singletons[label]

    @classmethod
    def key_set(cls):
        return set(cls.singletons.keys())

In [ ]:
s = Shareds.get("foo")


def update(d: dict):
    d["a"] = 1
    return 1


result = s.apply(update)
print(result, s.copy_data())

In [ ]:
from whendo.core.action import Action
from typing import List
from enum import Enum

class ListOpMode(Enum):
    ALL = 'all'
    OR = 'or'
    AND = 'and'

class ListAction(Action)
    op_mode:ListOpMode
    action_list:List[Action]:[]

    def execute(self, tag:str=None, scheduler_info:dict=None):
        processing_count, success_count, failure_count, successful_actions, exception_actions = process_action_list(
            op_mode=self.op_mode,
            action_list=self.action_list
            )
        if success_count == 0:
            return Exception(f"no action succeeded")
        else:
            processing_info = {
                'processing_count':processing_count,
                'success_count':success_count,
                'exception_count':failure_count,
                'successful_actions':successful_actions,
                'exception_actions':exception_actions
            }
            return {'outcome':'list action executed', 'action':self.info(), 'processing_info':processing_info}

class IfElseAction(Action):
    op_mode:ListOpMode
    test_action:Action
    if_actions:List[Action]
    else_action:Action

    def execute(self, tag:str=None, scheduler_info:dict=None):
        test_result = self.test_action.execute(tag=tag, scheduler_info=scheduler_info)
        processing_count = 1
        success_count = 0
        exception_count = 0
        successful_actions = []
        exception_actions = []
        else_test = isinstance(test_result, Exception)
        if else_test: # execute the else action
            exception_count = 1
            exception_actions.append(self.test_action.dict())

            else_result = else_action.execute(tag=tag, scheduler_info=scheduler_info)
            processing_count += 1
            if isinstance(else_result, Exception):
                exception_count += 1
                exception_actions.append(self.else_action.dict())
            else:
                success_count += 1
                successful_actions.append(self.else_action.dict())
        else:
            success_count += 1
            successful_actions.append(self.test_action.dict())
            processing_count, success_count, failure_count, successful_actions, exception_actions = process_action_list(
                op_mode=self.op_mode,
                action_list=self.if_actions,
                processing_count=processing_count,
                success_count=success_count,
                exception_count=exception_count,
                successful_actions=successful_actions,
                exception_actions=exception_actions
            )
        if success_count == 0:
            return Exception(f"no action succeeded")
        else:
            processing_info = {
                'else_test':else_test,
                'processing_count':processing_count,
                'success_count':success_count,
                'exception_count':failure_count,
                'successful_actions':successful_actions,
                'exception_actions':exception_actions
            }
            return {'outcome':'if-else action executed', 'action':self.info(), 'processing_info':processing_info}

def process_action_list(
    op_mode:ListOpMode,
    action_list:List[Action],
    processing_count:int=0,
    success_count:int=0,
    exception_count:int=0,
    successful_actions:List[Action]=[],
    exception_actions:List[Action]=[]
    ):
    processing_count = processing_count
    success_count = success_count
    exception_count = exception_count
    successful_actions = successful_actions
    exception_actions = exception_actions
    if op_mode == ListOpMode.ALL: # invoke all regardless of outcomes
        for action in action_list:
            result = action.execute(tag=tag, scheduler_info=scheduler_info)
            processing_count += 1
            if isinstance(result, Exception):
                exception_count += 1
                exception_actions.append(action.dict())
            else:
                success_count += 1
                successful_actions.append(action.dict())
    elif op_mode == ListOpMode.OR: # stop after first success
        for action in action_list:
            result = action.execute(tag=tag, scheduler_info=scheduler_info)
            processing_count += 1
            if isinstance(result, Exception):
                exception_count += 1
                exception_actions.append(action.dict())
            else:
                success_count += 1
                successful_actions.append(action.dict())
                break
    elif op_mode == ListOpMode.AND: # stop after first failure
        for action in action_list:
            action.execute(tag=tag, scheduler_info=scheduler_info)
            processing_count += 1
            if isinstance(result, Exception):
                exception_count += 1
                exception_actions.append(action.dict())
                break
            else:
                success_count += 1
                successful_actions.append(action.dict())
    return processing_count, success_count, failure_count, successful_actions, exception_actions
    }



### architecture thoughts

- 100% container based deployment
    - multiple containers per pi, distinguished by port
        - this could be a way of restoring previous behavior at a pi
    - container synonymous with mothership
- some machines (not necessarily pi's) could host multiple motherships 
    - these could be repositories for libraries of motherships, to be used as templates for pi's in the field
- a fleet is a collection of motherships
    - perhaps the fleet api would listen on 5050
        - the others listening on some other port
    - maybe an associated restful api that manages the fleet (a collection of mothership hosts)
    - there is a mothership sdk. there will be a fleet sdk.
    - actions would include
        - use a mothership's configuration to replace the configurations in other motherships
    - there would be a mothership discovery function
        - this would allow any machine to take on the role of fleet manager (taking fleet api requests)
        - plus a mothership health monitoring function


In [ ]:
import json

e = ["ack!", "kack", "whack"]
isinstance(e, Exception)

In [ ]:
class Local:
    client:Client=Client()

class Action(BaseModel):
    """
    Actions get something done.
    """

    def execute(self, tag: str = None):
    def execute(self, tag: str = None, action:Optional[Action]=None) -> Optional[Action]=None:
        """
        This method attempts to do something useful and return something useful
        """
        return Action or None

    def info(self):
        return object_info(self)

    def flat(self):
        return self.json()
    
class Payload(Action):
    """
    usage:

        p = Payload(payload=some_dictionary)
        action.execute(action=p)

        Local.client.execute_action('foo', p)

    """
    payload:Dict[str, Any]

    def execute(self, ...):
        return payload

class FilePath(Action):
    file:str

    def execute(self, ...):
        return file

class ClientAction(Action):
    host:str
    port:int

    def execute(self, ...):
        return Client(host=self.host, port=self.port)

class AddAction(Action):



In [ ]:
import whendo.core.action as action
import whendo.sdk.client as client
import whendo.core.util as util

class GetInfo(Action):
    some_field:str

    def execute(self, tag: str = None, scheduler_info: Dict[str, Any] = None):
        c = client.Client(host=blah, port=8000)
        solar_info_action = c.execute_action('get_solar_info')
        solar_info = solar_info_action.execute()
        shared_solar = util.SharedRW.get('solar_info')

class Payload(Action):
    data:Dict[str, Any]
    def execute(...):
        return data


In [ ]:
[1, 2, 3][-1:]

In [ ]:
import pathlib
import os


class Dirs:
    """
    This class computes directory paths for saved, output and log files. It creates the directories if absent.

    {home dir} / .whendo / cwd directory name / label /

    For example, if /home/pi/dev/whatnot/ were the current working directory, the computed directory with
    a label of 'output' would be /home/pi/.whendo/whatnot/output.

    This allows multiple whendo api servers running on a computer. That said, its important to give some thought to
    where you choose to initially run the server. You can run the server from another directory if you remember
    to move the relevant subdirectory of [{home dir}/.whendo/].

    """

    @classmethod
    def output_dir(cls, home_path: pathlib.Path = None):
        return str(cls.assure_dir(cls.dir_from_label("output", home_path)))

    @classmethod
    def saved_dir(cls, home_path: pathlib.Path = None):
        return str(cls.assure_dir(cls.dir_from_label("saved", home_path)))

    @classmethod
    def log_dir(cls, home_path: pathlib.Path = None):
        return str(cls.assure_dir(cls.dir_from_label("log", home_path)))

    @classmethod
    def dir_from_label(cls, label: str, home_path: pathlib.Path = None):
        path = home_path if home_path else Path.home()
        return str(path / ".whendo" / os.getcwd().split("/")[-1:][0] / label) + "/"

    @classmethod
    def assure_dir(cls, assured_dir: str):
        if not os.path.exists(assured_dir):
            os.makedirs(assured_dir)
        return assured_dir

In [ ]:
Dirs.log_dir(home_path=pathlib.Path("a/b/c"))

In [ ]:
os.path.join(util.Dirs.output_dir(), "job.log")

In [ ]:
pathlib.Path.home() / "foo"

In [ ]:
pathlib.Path.home()

In [ ]:
str(pathlib.Path.home() / "foo") + "/"

## event abstraction

In [ ]:
class Callback(BaseModel):
    action: Action

    def call(self, channel: int):
        action.execute()

In [ ]:
import Mock.GPIO as GPIO

GPIO.RISING

In [ ]:
class Channel(BaseModel):
    pin:int
    events:set(int)
    actions:[Action]

    def add_event(self, event:int):
        events.add(event)
    
    def add_action(self, action:Action):
        actions

def add_callback(self, channel_name:str, channel:Channel):
    pass

def attach_action(self, channel_name:str, action_name:str):
    pass

def activate_callback(self, channel:str):
    pass


    

class RiserCallback(BaseModel):
    channel:int
    event:int
    initialized:False

    def initialize(self):
        if not self.initialized:
            GPIO.add_event_detect(self.channel, self.event)
            self.initialized = True

    def terminate(self):
        if self.initialized:
            GPIO.remove_event_detect(self.channel)


    def callback_action(self, action:Action):
        def foo():
            action.execute
        GPIO.add_event_callback(self.channel, action)
    



Dispatcher


    
    def initiate_callbacks(self):
        for action_name in self.action_names:
            def function(channel:int):
                return actions[action_name].execute(data=)
            GPIO.add_event_callback(self.channel, action)
    
    def terminate_callbacks(self):
        GPIO.remove_event_detect(self.pin)
